In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/Prob_Stats_for_Data_Science/ProbStats Project/'

/content/gdrive/MyDrive/Prob_Stats_for_Data_Science/ProbStats Project


In [3]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<b><h1> Q-2 Part-D </h1></b>

In [4]:
cleaned_mt = pd.read_csv('vaccination_MT_cleaned.csv')
cleaned_mt

,Date,Location,Distributed
0,2020-12-14,MT,1950
1,2020-12-15,MT,7800
2,2020-12-16,MT,0
3,2020-12-17,MT,0
4,2020-12-18,MT,0
...,...,...,...
467,2022-05-10,MT,1100
468,2022-05-11,MT,400
469,2022-05-12,MT,2300
470,2022-05-13,MT,2700


In [5]:
cleaned_mn = pd.read_csv('vaccination_MN_cleaned.csv')
cleaned_mn

,Date,Location,Distributed
0,2020-12-14,MN,1950
1,2020-12-15,MN,4875
2,2020-12-16,MN,0
3,2020-12-17,MN,39975
4,2020-12-18,MN,0
...,...,...,...
478,2022-05-10,MN,11800
479,2022-05-11,MN,6900
480,2022-05-12,MN,26200
481,2022-05-13,MN,16200


**<h1> AutoRegression </h1>**

In [6]:
# Creeating matrices for multiple linear regression
def Create_Matrix(data, p):
    X = []
    Y = []
    for i in range(len(data) - p):
      X.append([1] + data[i:i+p].tolist())
      Y.append(data[i + p])
    return np.array(X), np.array(Y)

# Making predictions using AutoRegression
def AutoRegression(data, p):
    X, Y = Create_Matrix(data[:-7], p)
    beta = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(Y)
    y_predicted = []
    for i in range(len(data) - 7, len(data)):
        predict_value = data[i - p:i]
        predict_value = np.array([1] + predict_value.tolist())
        x = np.dot(beta, predict_value)
        y_predicted.append(x)
        predict_value = np.reshape(predict_value, (1, p + 1))
        X = np.append(X, list(predict_value), axis=0)
        Y = np.append(Y, data[i])
        beta = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(Y)
    return y_predicted

**<h1>EWMA</h1>**

In [7]:
def Calculate_EWMA(data, alpha):
    y_predicted = []
    y_predicted.append(data[0])
    for i in range(1,len(data)):
        y_predicted.append(alpha * data[i-1] + (1 - alpha) * y_predicted[i-1])
    return y_predicted

**<h1>MAPE & MSE</h1>**

In [8]:
def Calculate_MAPE(y_actual, y_pred):
    mape = 0
    for i in range(0,len(y_actual)):
        mape += (abs(y_actual[i] - y_pred[i])/y_actual[i])
    return (mape/len(y_actual))*100
    
def Calculate_MSE(y_actual, y_pred):
    mse = 0
    for i in range(0,len(y_actual)):
        mse += (np.square(y_pred[i] - y_actual[i]))
    return mse/len(y_actual)

<b><h2>May Data (State - Montana)</h2></b>

In [9]:
start_date = '2021-05-01' 
end_date = '2021-05-28'
date_selection = (cleaned_mt['Date'] >= start_date) & (cleaned_mt['Date'] <= end_date)
may_4_weeks_mt = cleaned_mt.loc[date_selection]
may_4_weeks_mt = may_4_weeks_mt.reset_index(drop=True)
may_4_weeks_mt

,Date,Location,Distributed
0,2021-05-01,MT,11330
1,2021-05-02,MT,0
2,2021-05-03,MT,0
3,2021-05-04,MT,3770
4,2021-05-05,MT,3140
5,2021-05-06,MT,6420
6,2021-05-07,MT,2750
7,2021-05-08,MT,8640
8,2021-05-09,MT,0
9,2021-05-10,MT,0


**<h1> Results - Montana State</h1>**

In [10]:
result_2_d_mt = pd.DataFrame()
result_2_d_mt['Date'] = may_4_weeks_mt['Date'][-7:]
result_2_d_mt['Location'] = may_4_weeks_mt['Location'][-7:]
result_2_d_mt['Distributed'] = may_4_weeks_mt['Distributed'][-7:]
result_2_d_mt['AR(3)'] = AutoRegression(may_4_weeks_mt['Distributed'], 3)
result_2_d_mt['AR(5)'] = AutoRegression(may_4_weeks_mt['Distributed'], 5)
result_2_d_mt['EWMA(0.5)'] = Calculate_EWMA(may_4_weeks_mt['Distributed'], 0.5)[-7:]
result_2_d_mt['EWMA(0.8)'] = Calculate_EWMA(may_4_weeks_mt['Distributed'], 0.8)[-7:]
result_2_d_mt = result_2_d_mt.reset_index(drop=True)
result_2_d_mt

,Date,Location,Distributed,AR(3),AR(5),EWMA(0.5),EWMA(0.8)
0,2021-05-22,MT,4040,1919.736253,2092.733234,3353.995676,2660.869017
1,2021-05-23,MT,0,3555.632740,3871.466664,3696.997838,3764.173803
2,2021-05-24,MT,0,4610.000755,3547.052082,1848.498919,752.834761
3,2021-05-25,MT,700,4684.057950,5961.109748,924.249460,150.566952
4,2021-05-26,MT,4580,4762.076997,6917.908770,812.124730,590.113390
5,2021-05-27,MT,3700,4239.534110,5551.325916,2696.062365,3782.022678
6,2021-05-28,MT,3610,3309.423608,4927.941954,3198.031182,3716.404536


**<h3> Removing rows where distribution of vaccine is zero becuase it will give division by zero error while calculating MAPE</h3>**

In [11]:
result_2_d_mt = result_2_d_mt[result_2_d_mt.Distributed != 0]
result_2_d_mt

,Date,Location,Distributed,AR(3),AR(5),EWMA(0.5),EWMA(0.8)
0,2021-05-22,MT,4040,1919.736253,2092.733234,3353.995676,2660.869017
3,2021-05-25,MT,700,4684.057950,5961.109748,924.249460,150.566952
4,2021-05-26,MT,4580,4762.076997,6917.908770,812.124730,590.113390
5,2021-05-27,MT,3700,4239.534110,5551.325916,2696.062365,3782.022678
6,2021-05-28,MT,3610,3309.423608,4927.941954,3198.031182,3716.404536


In [12]:
mse_mt_ar_3 = Calculate_MSE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['AR(3)'].tolist())
mse_mt_ar_5 = Calculate_MSE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['AR(5)'].tolist())
mse_mt_ewma_5 = Calculate_MSE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['EWMA(0.5)'].tolist())
mse_mt_ewma_8 = Calculate_MSE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['EWMA(0.8)'].tolist())
print("MSE for Vaccination in Montana State with AutoRegression(3) is ", mse_mt_ar_3)
print("MSE for Vaccination in Montana State with AutoRegression(5) is ", mse_mt_ar_5)
print("MSE for Vaccination in Montana State with EWMA(0.5) is ", mse_mt_ewma_5)
print("MSE for Vaccination in Montana State with EWMA(0.8) is ", mse_mt_ewma_8)

MSE for Vaccination in Montana State with AutoRegression(3) is  4156566.273026713
MSE for Vaccination in Montana State with AutoRegression(5) is  8420263.939006817
MSE for Vaccination in Montana State with EWMA(0.5) is  3179076.577316552
MSE for Vaccination in Montana State with EWMA(0.8) is  3628224.748761411


In [13]:
mape_mt_ar_3 = Calculate_MAPE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['AR(3)'].tolist())
mape_mt_ar_5 = Calculate_MAPE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['AR(5)'].tolist())
mape_mt_ewma_5 = Calculate_MAPE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['EWMA(0.5)'].tolist())
mape_mt_ewma_8 = Calculate_MAPE(result_2_d_mt['Distributed'].tolist(), result_2_d_mt['EWMA(0.8)'].tolist())
print("MAPE for Vaccination in Montana State with AutoRegression(3) is ", mape_mt_ar_3)
print("MAPE for Vaccination in Montana State with AutoRegression(5) is ", mape_mt_ar_5)
print("MAPE for Vaccination in Montana State with EWMA(0.5) is ", mape_mt_ewma_5)
print("MAPE for Vaccination in Montana State with EWMA(0.8) is ", mape_mt_ewma_8)

MAPE for Vaccination in Montana State with AutoRegression(3) is  129.70332215458674
MAPE for Vaccination in Montana State with AutoRegression(5) is  187.47534904433886
MAPE for Vaccination in Montana State with EWMA(0.5) is  33.96585786448997
MAPE for Vaccination in Montana State with EWMA(0.8) is  40.98141845554573


<b><h2>May Data (State - Minnesota)</h2></b>

In [14]:
start_date = '2021-05-01' 
end_date = '2021-05-28'
date_selection = (cleaned_mn['Date'] >= start_date) & (cleaned_mn['Date'] <= end_date)
may_4_weeks_mn = cleaned_mn.loc[date_selection]
may_4_weeks_mn = may_4_weeks_mn.reset_index(drop=True)
may_4_weeks_mn

,Date,Location,Distributed
0,2021-05-01,MN,57260
1,2021-05-02,MN,0
2,2021-05-03,MN,0
3,2021-05-05,MN,81240
4,2021-05-06,MN,19900
5,2021-05-07,MN,33530
6,2021-05-08,MN,37160
7,2021-05-09,MN,0
8,2021-05-10,MN,0
9,2021-05-13,MN,35390


**<h1> Results - Minnesota State</h1>**

In [15]:
result_2_d_mn = pd.DataFrame()
result_2_d_mn['Date'] = may_4_weeks_mn['Date'][-7:]
result_2_d_mn['Location'] = may_4_weeks_mn['Location'][-7:]
result_2_d_mn['Distributed'] = may_4_weeks_mn['Distributed'][-7:]
result_2_d_mn['AR(3)'] = AutoRegression(may_4_weeks_mn['Distributed'], 3)
result_2_d_mn['AR(5)'] = AutoRegression(may_4_weeks_mn['Distributed'], 5)
result_2_d_mn['EWMA(0.5)'] = Calculate_EWMA(may_4_weeks_mn['Distributed'], 0.5)[-7:]
result_2_d_mn['EWMA(0.8)'] = Calculate_EWMA(may_4_weeks_mn['Distributed'], 0.8)[-7:]
result_2_d_mn = result_2_d_mn.reset_index(drop=True)
result_2_d_mn

,Date,Location,Distributed,AR(3),AR(5),EWMA(0.5),EWMA(0.8)
0,2021-05-22,MN,39910,7622.897900,7515.170815,45528.381042,53287.467034
1,2021-05-23,MN,0,20044.748810,-13106.736563,42719.190521,42585.493407
2,2021-05-24,MN,0,34170.050198,19843.012802,21359.595261,8517.098681
3,2021-05-25,MN,83380,45974.250878,26889.300169,10679.797630,1703.419736
4,2021-05-26,MN,25020,28621.451962,22474.937841,47029.898815,67044.683947
5,2021-05-27,MN,11920,9403.044365,19474.323655,36024.949408,33424.936789
6,2021-05-28,MN,12820,30214.124963,33940.552499,23972.474704,16220.987358


**<h3> Removing rows where distribution of vaccine is zero becuase it will give division by zero error while calculating MAPE</h3>**

In [16]:
result_2_d_mn = result_2_d_mn[result_2_d_mn.Distributed != 0]
result_2_d_mn

,Date,Location,Distributed,AR(3),AR(5),EWMA(0.5),EWMA(0.8)
0,2021-05-22,MN,39910,7622.897900,7515.170815,45528.381042,53287.467034
3,2021-05-25,MN,83380,45974.250878,26889.300169,10679.797630,1703.419736
4,2021-05-26,MN,25020,28621.451962,22474.937841,47029.898815,67044.683947
5,2021-05-27,MN,11920,9403.044365,19474.323655,36024.949408,33424.936789
6,2021-05-28,MN,12820,30214.124963,33940.552499,23972.474704,16220.987358


In [17]:
mse_mn_ar_3 = Calculate_MSE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['AR(3)'].tolist())
mse_mn_ar_5 = Calculate_MSE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['AR(5)'].tolist())
mse_mn_ewma_5 = Calculate_MSE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['EWMA(0.5)'].tolist())
mse_mn_ewma_8 = Calculate_MSE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['EWMA(0.8)'].tolist())
print("MSE for Vaccination in Minnesota State with AutoRegression(3) is ", mse_mn_ar_3)
print("MSE for Vaccination in Minnesota State with AutoRegression(5) is ", mse_mn_ar_5)
print("MSE for Vaccination in Minnesota State with EWMA(0.5) is ", mse_mn_ewma_5)
print("MSE for Vaccination in Minnesota State with EWMA(0.8) is ", mse_mn_ewma_8)

MSE for Vaccination in Minnesota State with AutoRegression(3) is  552701626.9075754
MSE for Vaccination in Minnesota State with AutoRegression(5) is  950049402.0830244
MSE for Vaccination in Minnesota State with EWMA(0.5) is  1301349510.7892516
MSE for Vaccination in Minnesota State with EWMA(0.8) is  1818024694.002908


In [18]:
mape_mn_ar_3 = Calculate_MAPE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['AR(3)'].tolist())
mape_mn_ar_5 = Calculate_MAPE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['AR(5)'].tolist())
mape_mn_ewma_5 = Calculate_MAPE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['EWMA(0.5)'].tolist())
mape_mn_ewma_8 = Calculate_MAPE(result_2_d_mn['Distributed'].tolist(), result_2_d_mn['EWMA(0.8)'].tolist())
print("MAPE for Vaccination in Minnesota State with AutoRegression(3) is ", mape_mn_ar_3)
print("MAPE for Vaccination in Minnesota State with AutoRegression(5) is ", mape_mn_ar_5)
print("MAPE for Vaccination in Minnesota State with EWMA(0.5) is ", mape_mn_ewma_5)
print("MAPE for Vaccination in Minnesota State with EWMA(0.8) is ", mape_mn_ewma_8)

MAPE for Vaccination in Minnesota State with AutoRegression(3) is  59.39017000731837
MAPE for Vaccination in Minnesota State with AutoRegression(5) is  77.44296257089414
MAPE for Vaccination in Minnesota State with EWMA(0.5) is  95.69075502666378
MAPE for Vaccination in Minnesota State with EWMA(0.8) is  101.27595925528125


**<h1>Part2-E</h1>**

**<h2>Paired T-test for September 2021 between two States</h2>**

---



In [19]:
start_date = '2021-09-01' 
end_date = '2021-09-30'
date_selection = (cleaned_mt['Date'] >= start_date) & (cleaned_mt['Date'] <= end_date)
sep_mt = cleaned_mt.loc[date_selection]
sep_mt = sep_mt.reset_index(drop=True)
date_selection = (cleaned_mn['Date'] >= start_date) & (cleaned_mn['Date'] <= end_date)
sep_mn = cleaned_mn.loc[date_selection]
sep_mn = sep_mn.reset_index(drop=True)

In [20]:
result_2_e_sep = pd.DataFrame()
result_2_e_sep['Date'] = sep_mt['Date']
result_2_e_sep['Vaccine_MN'] = sep_mn['Distributed']
result_2_e_sep['Vaccine_MT'] = sep_mt['Distributed']
result_2_e_sep['Difference'] = result_2_e_sep['Vaccine_MN'] - result_2_e_sep['Vaccine_MT']
result_2_e_sep = result_2_e_sep.reset_index(drop=True)
result_2_e_sep

,Date,Vaccine_MN,Vaccine_MT,Difference
0,2021-09-01,24640,1890,22750
1,2021-09-02,22150,4750,17400
2,2021-09-03,47130,5070,42060
3,2021-09-04,40910,2840,38070
4,2021-09-05,-100,-100,0
5,2021-09-06,0,0,0
6,2021-09-07,0,0,0
7,2021-09-08,7020,0,7020
8,2021-09-09,23980,1440,22540
9,2021-09-10,13920,5140,8780


In [21]:
def Calculate_std(x, mean_x):
  square_sum=0
  for i in range(0, len(x)):
    square_sum += (x[i] - mean_x)**2
  return np.sqrt(square_sum / len(x))

In [22]:
def Calculate_Paired_T_test(mean_d, std_d, len_d):
  return mean_d / (std_d / np.sqrt(len_d))

In [23]:
len_sep = len(result_2_e_sep['Difference'])
mean_sep = result_2_e_sep['Difference'].mean()
std_sep = Calculate_std(result_2_e_sep['Difference'], mean_sep)
T = Calculate_Paired_T_test(mean_sep, std_sep, len_sep)
t_value = 2.0423 # t(29, 0.025)
if abs(T) > t_value:
  print("Paired t-test for mean of vaccine distribution between Minnesota and Montana is", abs(T), "which is greater than t_value(2.043) hence reject null hypothesis")
else:
  print("Paired t-test for mean of vaccine distribution between Minnesota and Montana is", abs(T), " which is not greater than t_value(2.043) hence accept null hypothesis")

Paired t-test for mean of vaccine distribution between Minnesota and Montana is 5.448478346062849 which is greater than t_value(2.043) hence reject null hypothesis


**<h2>Paired T-test for November 2021 between two States</h2>**

In [24]:
start_date = '2021-11-01' 
end_date = '2021-11-30'
date_selection = (cleaned_mt['Date'] >= start_date) & (cleaned_mt['Date'] <= end_date)
nov_mt = cleaned_mt.loc[date_selection]
nov_mt = nov_mt.reset_index(drop=True)
date_selection = (cleaned_mn['Date'] >= start_date) & (cleaned_mn['Date'] <= end_date)
nov_mn = cleaned_mn.loc[date_selection]
nov_mn = nov_mn.reset_index(drop=True)

In [25]:
nov_mn['Date'] = pd.to_datetime(nov_mn['Date'])
nov_mn

,Date,Location,Distributed
0,2021-11-01,MN,-2200
1,2021-11-02,MN,14470
2,2021-11-04,MN,80660
3,2021-11-05,MN,22440
4,2021-11-06,MN,33640
5,2021-11-07,MN,12160
6,2021-11-08,MN,0
7,2021-11-09,MN,45980
8,2021-11-11,MN,58510
9,2021-11-12,MN,72160


In [26]:
nov_mt['Date'] = pd.to_datetime(nov_mt['Date'])
nov_mt

,Date,Location,Distributed
0,2021-11-01,MT,-200
1,2021-11-05,MT,6820
2,2021-11-06,MT,3200
3,2021-11-07,MT,1820
4,2021-11-08,MT,0
5,2021-11-09,MT,6500
6,2021-11-10,MT,12330
7,2021-11-11,MT,9200
8,2021-11-13,MT,5790
9,2021-11-14,MT,0


**<h3>As some data is removed in outlier, which makes data from both states imbalanced. Thus, 0 is added assuming that vaccine distribution on that day would be zero in that particular state. </h3>**

In [27]:
result_2_e_nov = pd.DataFrame()
result_2_e_nov['Date'] = pd.date_range(start='11/1/2021', periods=30, freq='D')
result_2_e_nov = pd.merge(result_2_e_nov,nov_mn,on='Date',how='left')
result_2_e_nov = result_2_e_nov.rename(columns={'Distributed': 'Vaccine_MN'})
result_2_e_nov.drop('Location', axis=1, inplace=True)
result_2_e_nov = pd.merge(result_2_e_nov,nov_mt,on='Date',how='left')
result_2_e_nov = result_2_e_nov.rename(columns={'Distributed': 'Vaccine_MT'})
result_2_e_nov.drop('Location', axis=1, inplace=True)
result_2_e_nov = result_2_e_nov.fillna(0)
result_2_e_nov['Difference'] = result_2_e_nov['Vaccine_MN'] - result_2_e_nov['Vaccine_MT']
result_2_e_nov = result_2_e_nov.reset_index(drop=True)
result_2_e_nov

,Date,Vaccine_MN,Vaccine_MT,Difference
0,2021-11-01,-2200.0,-200.0,-2000.0
1,2021-11-02,14470.0,0.0,14470.0
2,2021-11-03,0.0,0.0,0.0
3,2021-11-04,80660.0,0.0,80660.0
4,2021-11-05,22440.0,6820.0,15620.0
5,2021-11-06,33640.0,3200.0,30440.0
6,2021-11-07,12160.0,1820.0,10340.0
7,2021-11-08,0.0,0.0,0.0
8,2021-11-09,45980.0,6500.0,39480.0
9,2021-11-10,0.0,12330.0,-12330.0


In [28]:
len_nov = len(result_2_e_nov['Difference'])
mean_nov = result_2_e_nov['Difference'].mean()
std_nov = Calculate_std(result_2_e_nov['Difference'], mean_sep)
T = Calculate_Paired_T_test(mean_nov, std_nov, len_nov)
t_value = 2.0423 # t(29, 0.025)
if abs(T) > t_value:
  print("Paired t-test for mean of vaccine distribution between Minnesota and Montana is", abs(T), "which is greater than t_value(2.043) hence reject null hypothesis")
else:
  print("Paired t-test for mean of vaccine distribution between Minnesota and Montana is", abs(T), " which is not greater than t_value(2.043) hence accept null hypothesis")

Paired t-test for mean of vaccine distribution between Minnesota and Montana is 3.728625815262307 which is greater than t_value(2.043) hence reject null hypothesis
